# List of Exposures with Hologram at USDF


- work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2022/10/31



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2022_39/summit_utils/python', '/home/d/dagoret/repos/repos_w_2022_39/drp_pipe/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_tools/gb9b489fd42+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_drp/g12d5fbe6a5+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/faro/gbb36616a4b+5e2d81108f/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcmcal/g5147cf65fe+deeae05922/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcm/g3dbc634184/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/jointcal/ge7491f621d+ceb865f59e/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_cfht/gd4e7b81488+e2797d1921/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_39
   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [9]:
collection='LATISS/raw/all'

## List of Exposures

In [10]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [11]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [12]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2022083100004,AT_O_20220831_000004,20220831,4,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
1,2022083100008,AT_O_20220831_000008,20220831,8,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
2,2022083100007,AT_O_20220831_000007,20220831,7,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
3,2022083100009,AT_O_20220831_000009,20220831,9,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
4,2022083100012,AT_O_20220831_000012,20220831,12,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
27875,2022120800520,AT_O_20221208_000520,20221208,520,science,SDSS-J081508_057,SDSSr_65mm~empty,38.114088,30.0,123.40728,7.811391,179.999181
27876,2022120800522,AT_O_20221208_000522,20221208,522,science,SDSS-J081508_058,SDSSr_65mm~empty,38.151061,30.0,123.502148,7.811367,87.224188
27877,2022120800524,AT_O_20221208_000524,20221208,524,science,SDSS-J081508_058,SDSSg_65mm~empty,38.222868,30.0,123.502108,7.811388,179.998862
27878,2022120800526,AT_O_20221208_000526,20221208,526,science,SDSS-J081508_058,SDSSi_65mm~empty,38.262094,30.0,123.502129,7.811301,179.998821


## Selection of science

In [13]:
df_science = df_exposure[df_exposure.type == 'science']

In [14]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_9128/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [15]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
27801,2022120800469,AT_O_20221208_000469,20221208,469,science,SDSS-J081508_049,SDSSi_65mm~empty,38.588491,30.0,123.407455,7.717392,275.137463
27828,2022120800470,AT_O_20221208_000470,20221208,470,science,SDSS-J081508_049,SDSSg_65mm~empty,38.522126,30.0,123.407423,7.717389,180.001133
27802,2022120800471,AT_O_20221208_000471,20221208,471,science,SDSS-J081508_049,SDSSg_65mm~empty,38.497359,30.0,123.407334,7.7174,180.001163
27829,2022120800472,AT_O_20221208_000472,20221208,472,science,SDSS-J081508_049,SDSSr_65mm~empty,38.465749,30.0,123.407358,7.717393,180.001203
27803,2022120800473,AT_O_20221208_000473,20221208,473,science,SDSS-J081508_049,SDSSr_65mm~empty,38.441995,30.0,123.407422,7.717379,180.001211
27830,2022120800474,AT_O_20221208_000474,20221208,474,science,SDSS-J081508_050,SDSSr_65mm~empty,38.376674,30.0,123.502295,7.717391,272.921612
27804,2022120800475,AT_O_20221208_000475,20221208,475,science,SDSS-J081508_050,SDSSr_65mm~empty,38.355011,30.0,123.502178,7.71739,272.921471
27831,2022120800476,AT_O_20221208_000476,20221208,476,science,SDSS-J081508_050,SDSSg_65mm~empty,38.302094,30.0,123.502204,7.717387,180.00074
27805,2022120800477,AT_O_20221208_000477,20221208,477,science,SDSS-J081508_050,SDSSg_65mm~empty,38.281627,30.0,123.50223,7.717428,180.001024
27832,2022120800478,AT_O_20221208_000478,20221208,478,science,SDSS-J081508_050,SDSSi_65mm~empty,38.254928,30.0,123.502262,7.717379,180.00099


In [16]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [17]:
#date_sel = 20220630
#date_sel = 20220630
date_sel = 20221208

In [18]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [19]:
len(df_science_selected)

331

# List of filters

In [20]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['SDSSr_65mm~empty', 'SDSSg_65mm~empty', 'SDSSi_65mm~empty',
       'empty~holo4_003'], dtype=object)

In [21]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [22]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [23]:
pd.set_option('display.max_rows', None)

In [24]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20221208_filt_empty-holo4_003.list


/tmp/ipykernel_9128/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [25]:
len(all_def_science_selected_final)

1

In [28]:
len(all_def_science_selected_final[0])

84

In [26]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
27378,2022120800043,AT_O_20221208_000043,20221208,43,science,HD185975,empty~holo4_003,58.995908,30.0,307.073592,-87.514388,166.991719
27408,2022120800044,AT_O_20221208_000044,20221208,44,science,HD185975,empty~holo4_003,59.003646,30.0,307.07366,-87.514386,166.991658
27410,2022120800048,AT_O_20221208_000048,20221208,48,science,HD200654,empty~holo4_003,52.75803,30.0,316.623976,-50.006025,184.578137
27381,2022120800049,AT_O_20221208_000049,20221208,49,science,HD200654,empty~holo4_003,52.873245,30.0,316.623992,-50.006015,184.578043
27383,2022120800053,AT_O_20221208_000053,20221208,53,science,HD205905,empty~holo4_003,51.576477,30.0,324.769523,-27.343751,198.296079
27413,2022120800054,AT_O_20221208_000054,20221208,54,science,HD205905,empty~holo4_003,51.727959,30.0,324.769539,-27.343778,198.295953
27388,2022120800063,AT_O_20221208_000063,20221208,63,science,HD2811,empty~holo4_003,19.909766,30.0,7.846854,-43.647885,143.722396
27418,2022120800064,AT_O_20221208_000064,20221208,64,science,HD2811,empty~holo4_003,20.016259,30.0,7.846556,-43.647979,143.722936
27420,2022120800068,AT_O_20221208_000068,20221208,68,science,HD2811,empty~holo4_003,20.288583,30.0,7.845092,-43.648255,145.565943
27391,2022120800069,AT_O_20221208_000069,20221208,69,science,HD2811,empty~holo4_003,20.396357,30.0,7.845064,-43.648257,145.565962


In [27]:
all_def_science_selected_final[1]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[2]

In [ ]:
all_def_science_selected_final[3]

In [ ]:
all_def_science_selected_final[4]